In [1]:
import wrangle
import pandas as pd
import numpy as np
import 

SyntaxError: invalid syntax (784784744.py, line 4)

In [ ]:
df = wrangle.get_zillow_project_data()
df.head()

Let me get that for you...


,parcelid,bathroomcnt,bedroomcnt,finishedsquarefeet12,fips,garagecarcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,regionidcounty,yearbuilt,taxvaluedollarcnt,assessmentyear,logerror,transactiondate
0,17251466,2.0,4.0,2015.0,6111.0,2.0,34269929.0,-118750440.0,6500.0,1111,2061.0,1967.0,436209.0,2016.0,0.010302,2017-01-09
1,11944208,1.0,2.0,1063.0,6037.0,NaN,34083461.0,-118279154.0,3174.0,0100,3101.0,1941.0,535295.0,2016.0,-0.025546,2017-01-09
2,17267814,2.5,3.0,1937.0,6111.0,2.0,34194252.0,-118897518.0,20000.0,1110,2061.0,1960.0,629454.0,2016.0,-0.086395,2017-01-09
3,12192961,1.0,2.0,1058.0,6037.0,NaN,33975937.0,-118313850.0,4782.0,0102,3101.0,1926.0,300000.0,2016.0,-0.160147,2017-01-09
4,11720764,1.0,3.0,1244.0,6037.0,NaN,34023782.0,-118318304.0,5607.0,0100,3101.0,1910.0,259903.0,2016.0,-0.088564,2017-01-09


In [ ]:
df = wrangle.zillow_proj_prep(df)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51834 entries, 0 to 52441
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bathroomcnt                51834 non-null  float64
 1   bedroomcnt                 51834 non-null  float64
 2   finishedsquarefeet12       51834 non-null  float64
 3   fips                       51834 non-null  float64
 4   garagecarcnt               51834 non-null  int64  
 5   latitude                   51834 non-null  float64
 6   longitude                  51834 non-null  float64
 7   lotsizesquarefeet          51834 non-null  float64
 8   propertycountylandusecode  51834 non-null  object 
 9   regionidcounty             51834 non-null  float64
 10  yearbuilt                  51834 non-null  float64
 11  taxvaluedollarcnt          51834 non-null  float64
 12  assessmentyear             51834 non-null  float64
 13  logerror                   51834 non-null  flo

In [ ]:
# what are the data types of the columns?
df.dtypes

bathroomcnt                  float64
bedroomcnt                   float64
finishedsquarefeet12         float64
fips                         float64
garagecarcnt                   int64
latitude                     float64
longitude                    float64
lotsizesquarefeet            float64
propertycountylandusecode     object
regionidcounty               float64
yearbuilt                    float64
taxvaluedollarcnt            float64
assessmentyear               float64
logerror                     float64
transactiondate               object
dtype: object

In [ ]:
df.head()

,bathroomcnt,bedroomcnt,finishedsquarefeet12,fips,garagecarcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,regionidcounty,yearbuilt,taxvaluedollarcnt,assessmentyear,logerror,transactiondate
0,2.0,4.0,2015.0,6111.0,1,34269929.0,-118750440.0,6500.0,1111,2061.0,1967.0,436209.0,2016.0,0.010302,2017-01-09
1,1.0,2.0,1063.0,6037.0,0,34083461.0,-118279154.0,3174.0,0100,3101.0,1941.0,535295.0,2016.0,-0.025546,2017-01-09
2,2.5,3.0,1937.0,6111.0,1,34194252.0,-118897518.0,20000.0,1110,2061.0,1960.0,629454.0,2016.0,-0.086395,2017-01-09
3,1.0,2.0,1058.0,6037.0,0,33975937.0,-118313850.0,4782.0,0102,3101.0,1926.0,300000.0,2016.0,-0.160147,2017-01-09
4,1.0,3.0,1244.0,6037.0,0,34023782.0,-118318304.0,5607.0,0100,3101.0,1910.0,259903.0,2016.0,-0.088564,2017-01-09


In [ ]:
# split the dataframe into train, validate, and test
train, validate, test = wrangle.split_dataframe(df)
print(train.shape)
print(validate.shape)
print(test.shape)

(29026, 15)
(12441, 15)
(10367, 15)


In [ ]:
# create X and y for train, validate, and test
X_train = train.drop(columns=['taxvaluedollarcnt'])
y_train = train['taxvaluedollarcnt']
X_validate = validate.drop(columns=['taxvaluedollarcnt'])
y_validate = validate['taxvaluedollarcnt']
X_test = test.drop(columns=['taxvaluedollarcnt'])
y_test = test['taxvaluedollarcnt']

In [ ]:
# # # on the back burner
# # Identify and remove outliers
# # find the outliers in the training set
# X_train_no = prepare.remove_outliers_fibs(X_train, 1.5)
# X_train_no

In [ ]:
X_train

,bathroomcnt,bedroomcnt,finishedsquarefeet12,fips,garagecarcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,regionidcounty,yearbuilt,assessmentyear,logerror,transactiondate
48362,3.0,4.0,3318.0,6037.0,0,34138433.0,-118081972.0,14395.0,0101,3101.0,1932.0,2016.0,-0.003520,2017-08-31
9535,2.5,3.0,4741.0,6059.0,1,33881816.0,-117823351.0,18817.0,122,1286.0,1988.0,2016.0,-0.046878,2017-02-26
6490,2.0,3.0,1561.0,6059.0,1,33661392.0,-118004136.0,2925.0,122,1286.0,1978.0,2016.0,-0.035841,2017-02-23
8691,2.0,4.0,1366.0,6037.0,0,34104647.0,-117870328.0,8470.0,0100,3101.0,1955.0,2016.0,-0.013044,2017-03-01
5279,2.0,3.0,1244.0,6037.0,0,33958080.0,-118206458.0,6706.0,0100,3101.0,1955.0,2016.0,-0.038561,2017-02-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4670,2.0,4.0,1574.0,6037.0,0,34695130.0,-118108097.0,8341.0,0101,3101.0,1973.0,2016.0,-0.027050,2017-02-13
38241,2.0,4.0,1773.0,6037.0,0,34575619.0,-118010572.0,7285.0,0100,3101.0,2011.0,2016.0,0.028466,2017-07-13
19773,2.0,3.0,1174.0,6037.0,0,34089570.0,-117715481.0,6419.0,0100,3101.0,1952.0,2016.0,-0.009677,2017-04-20
51355,1.0,2.0,1161.0,6037.0,0,33846400.0,-118156511.0,4999.0,0100,3101.0,1952.0,2016.0,0.009994,2017-09-19


AttributeError: type object 'map' has no attribute 'view'